In [1]:
import pandas as pd
import numpy as np
from scipy import stats
%matplotlib inline
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
from IPython.display import Image
from sklearn import linear_model, datasets
from sklearn.metrics import mean_squared_error
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import folium
import pylab 
import scipy.stats as stats
from forex_python.converter import CurrencyRates
pd.set_option('display.max_columns', 40)

/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
c = CurrencyRates()
currenciesDict = {
    'BWP': 9.64, 
    'BRL': c.convert('USD', 'BRL', 1),
    'USD': c.convert('USD', 'USD', 1),
    'AED': 3.67, 
    'INR': c.convert('USD', 'INR', 1), 
    'IDR': c.convert('USD', 'IDR', 1), 
    'GBP': c.convert('USD', 'GBP', 1), 
    'QAR': 3.64, 
    'ZAR': c.convert('USD', 'ZAR', 1),
    'LKR': 155.04,
    'TRY': c.convert('USD', 'TRY', 1)
}
print(currenciesDict)
def convertCurrency(x):
    return x['Average Cost for two']/currenciesDict[x['Currency in short']]



{'BWP': 9.64, 'BRL': 3.4034, 'USD': 1.0, 'AED': 3.67, 'INR': 66.122, 'IDR': 13927.0, 'GBP': 0.71174, 'QAR': 3.64, 'ZAR': 12.025, 'LKR': 155.04, 'TRY': 4.0461}


In [3]:
df=pd.read_csv("zomato.csv",encoding = "ISO-8859-1",low_memory=False)

In [4]:
df.insert(loc=12,column="Currency in USD",value=0)
df.insert(loc=12,column="Currency in short",value=df["Currency"])
df["Currency in short"].replace("Botswana Pula(P)","BWP",inplace=True)
df["Currency in short"].replace("Brazilian Real(R$)","BRL",inplace=True)
df["Currency in short"].replace("Dollar($)","USD",inplace=True)
df["Currency in short"].replace("Emirati Diram(AED)","AED",inplace=True)
df["Currency in short"].replace("Indian Rupees(Rs.)","INR",inplace=True)
df["Currency in short"].replace("Indonesian Rupiah(IDR)","IDR",inplace=True)
df["Currency in short"].replace("NewZealand($)","USD",inplace=True)
df["Currency in short"].replace("Pounds(£)","GBP",inplace=True)
df["Currency in short"].replace("Qatari Rial(QR)","QAR",inplace=True)
df["Currency in short"].replace("Rand(R)","ZAR",inplace=True)
df["Currency in short"].replace("Sri Lankan Rupee(LKR)","LKR",inplace=True)
df["Currency in short"].replace("Turkish Lira(TL)","TRY",inplace=True)
df['Currency in USD'] = df.apply(convertCurrency, axis=1)

# Creating Cuisines as a feature

In [5]:
indian = ['Street Food','Vietnamese','Tibetan','North Eastern','Oriya','Parsi','Chettinad','Goan','Curry','Vegetarian','South Indian','Rajasthani','Pakistani','Lucknowi','Mughlai','Naga','Nepalese','Modern Indian','Mithai','Malwani', 'Mangalorean','Maharashtrian','Kashmiri','Kebab','Kerala','Indian','Hyderabadi','Gujarati','Burmese','Bihari', 'Biryani','Assamese', "Awadhi","Andhra", "North Indian", "Bengali" ]
asian = [ 'Thai','Teriyaki','Ramen','Indonesian','Dim Sum','Malaysian','Sushi', 'Taiwanese','Seafood','Singaporean','Korean','Japanese','Chinese','Cantonese','Bubble Tea','Asian','Asian Fusion',]
continental = [ 'Modern Australian','Moroccan','Belgian','Western', 'World Cuisine''Tapas','Southern', 'Southwestern', 'Spanish','Restaurant Cafe','Salad','Sandwich', 'Scottish','Pizza','Portuguese','Pub Food','New American','Irish','Italian','International','Diner','German', 'Gourmet Fast Food', 'Greek', 'Grill','Fish and Chips','French','European','Fast Food','Deli','Continental','Canadian','British','Bar Food','Burger','Australian','BBQ','American',]
middle_eastern = ['Brazilian','Turkish','Turkish Pizza','Middle Eastern','Lebanese','Iranian','Fusion','Afghani',]
latin = [ 'Tex-Mex','Tapas','South American','Mexican','Latin American','Filipino','Cuban','Argentine','Caribbean']
others = ['African', 'Arabian', 'Armenian', 'Bakery', 'Beverages', 'Breakfast', 'Bí_rek', 'Cafe', 'Cajun', 'Charcoal Grill', 'Coffee and Tea', 'Contemporary', 'Cuisine Varies', 'Desserts', 'Drinks Only', 'Durban', 'Dí_ner', 'Finger Food', 'Hawaiian', 'Healthy Food', 'Ice Cream', 'Izgara', 'Juices', 'Kiwi', 'Malay', 'Mediterranean', 'Mineira', 'Patisserie', 'Peranakan', 'Persian', 'Peruvian', 'Raw Meats', 'Soul Food', 'South African', 'Sri Lankan', 'Steak', 'Sunda', 'Tea', 'World Cuisine']

In [6]:
df.insert(loc=10,column="c_indian",value=0)
df.insert(loc=10,column="c_asian",value=0)
df.insert(loc=10,column="c_continental",value=0)
df.insert(loc=10,column="c_latin",value=0)
df.insert(loc=10,column="c_middle_eastern",value=0)
df.insert(loc=10,column="c_others",value=0)

In [7]:
def selectCuisine(item, c_list):
    if type(item['Cuisines']) is str:
        str_split = [x.strip() for x in item['Cuisines'].split(',')]
        if any(i in c_list for i in str_split):
            return 1
    return 0

In [8]:
df['c_indian'] = df.apply(selectCuisine,args=([indian]), axis=1)
df['c_asian'] = df.apply(selectCuisine,args=([asian]), axis=1)
df['c_continental'] = df.apply(selectCuisine,args=([continental]), axis=1)
df['c_latin'] = df.apply(selectCuisine,args=([latin]), axis=1)
df['c_middle_eastern'] = df.apply(selectCuisine,args=([middle_eastern]), axis=1)
df['c_others'] = df.apply(selectCuisine,args=([others]), axis=1)

# Relation between Aggregate rating, Rating color, Rating text

In [9]:
df_rrr = df.copy()

print(df_rrr['Rating color'].unique())
print(df_rrr['Rating text'].unique())

df_rrr['Rating color'].replace(to_replace=['Dark Green'], value=5, inplace=True)
df_rrr['Rating color'].replace(to_replace=['Green'], value=4, inplace=True)
df_rrr['Rating color'].replace(to_replace=['Yellow'], value=3, inplace=True)
df_rrr['Rating color'].replace(to_replace=['Orange'], value=2, inplace=True)
df_rrr['Rating color'].replace(to_replace=['Red'], value=1, inplace=True)
df_rrr['Rating color'].replace(to_replace=['White'], value=0, inplace=True)
df_rrr['Rating text'].replace(to_replace=['Excellent'], value=5, inplace=True)
df_rrr['Rating text'].replace(to_replace=['Very Good'], value=4, inplace=True)
df_rrr['Rating text'].replace(to_replace=['Good'], value=3, inplace=True)
df_rrr['Rating text'].replace(to_replace=['Average'], value=2, inplace=True)
df_rrr['Rating text'].replace(to_replace=['Poor'], value=1, inplace=True)
df_rrr['Rating text'].replace(to_replace=['Not rated'], value=0, inplace=True)

['Dark Green' 'Green' 'Yellow' 'Orange' 'White' 'Red']
['Excellent' 'Very Good' 'Good' 'Average' 'Not rated' 'Poor']


In [10]:
# Feature Correlation for Rating color, Rating text, Aggregate rating
feature_data = df_rrr[["Aggregate rating", "Rating color", "Rating text"]]
corr = feature_data.corr()
trace = go.Heatmap(z=np.array(corr), x=corr.columns, y=corr.columns)
data=[trace]
iplot(data)

#### Very high correlation among these, any of theses feature can be used to predict rating. For OLS Aggregate Rating must be used and Logistic Regression should be used for Rating text or Rating color

In [11]:
df['Rating color'].replace(to_replace=['Dark Green'], value=5, inplace=True)
df['Rating color'].replace(to_replace=['Green'], value=4, inplace=True)
df['Rating color'].replace(to_replace=['Yellow'], value=3, inplace=True)
df['Rating color'].replace(to_replace=['Orange'], value=2, inplace=True)
df['Rating color'].replace(to_replace=['Red'], value=1, inplace=True)
df['Rating color'].replace(to_replace=['White'], value=0, inplace=True)
df['Rating text'].replace(to_replace=['Excellent'], value=5, inplace=True)
df['Rating text'].replace(to_replace=['Very Good'], value=4, inplace=True)
df['Rating text'].replace(to_replace=['Good'], value=3, inplace=True)
df['Rating text'].replace(to_replace=['Average'], value=2, inplace=True)
df['Rating text'].replace(to_replace=['Poor'], value=1, inplace=True)
df['Rating text'].replace(to_replace=['Not rated'], value=0, inplace=True)

# Exploring dataset

In [12]:
df.describe() # for numerical data.

,Restaurant ID,Country Code,Longitude,Latitude,c_others,c_middle_eastern,c_latin,c_continental,c_asian,c_indian,Average Cost for two,Currency in USD,Price range,Aggregate rating,Rating color,Rating text,Votes
count,9.551000e+03,9551.000000,9551.000000,9551.000000,9551.000000,9551.000000,9551.000000,9551.000000,9551.000000,9551.000000,9551.000000,9551.000000,9551.000000,9551.000000,9551.000000,9551.000000,9551.000000
mean,9.051128e+06,18.365616,64.126574,25.854381,0.275050,0.016229,0.024605,0.380065,0.329599,0.537535,1199.210763,12.401683,1.804837,2.666370,2.071092,2.071092,156.909748
std,8.791521e+06,56.750546,41.467058,11.007935,0.446563,0.126360,0.154925,0.485428,0.470092,0.498615,16121.183073,19.682322,0.905609,1.516378,1.367335,1.367335,430.169145
min,5.300000e+01,1.000000,-157.948486,-41.330428,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,3.019625e+05,1.000000,77.081343,28.478713,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,250.000000,4.537068,1.000000,2.500000,2.000000,2.000000,5.000000
50%,6.004089e+06,1.000000,77.191964,28.570469,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,400.000000,7.561780,2.000000,3.200000,2.000000,2.000000,31.000000
75%,1.835229e+07,1.000000,77.282006,28.642758,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,700.000000,12.098848,2.000000,3.700000,3.000000,3.000000,131.000000
max,1.850065e+07,216.000000,174.832089,55.976980,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,800000.000000,622.406639,4.000000,4.900000,5.000000,5.000000,10934.000000


In [13]:
df.describe(include=['O']) # for text based data.

,Restaurant Name,City,Address,Locality,Locality Verbose,Cuisines,Currency,Currency in short,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu
count,9551,9551,9551,9551,9551,9542,9551,9551,9551,9551,9551,9551
unique,7446,141,8918,1208,1265,1825,12,11,2,2,2,1
top,Cafe Coffee Day,New Delhi,"Sector 41, Noida",Connaught Place,"Connaught Place, New Delhi",North Indian,Indian Rupees(Rs.),INR,No,No,No,No
freq,83,5473,11,122,122,936,8652,8652,8393,7100,9517,9551


# Final Data set

In [14]:
df.tail()

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,c_others,c_middle_eastern,c_latin,c_continental,c_asian,c_indian,Average Cost for two,Currency,Currency in short,Currency in USD,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
9546,5915730,NamlÛ± Gurme,208,ÛÁstanbul,"Kemankeô Karamustafa Paôa Mahallesi, RÛ±htÛ±...",Karakí_y,"Karakí_y, ÛÁstanbul",28.977392,41.022793,Turkish,0,1,0,0,0,0,80,Turkish Lira(TL),TRY,19.772126,No,No,No,No,3,4.1,4,4,788
9547,5908749,Ceviz AÛôacÛ±,208,ÛÁstanbul,"Koôuyolu Mahallesi, Muhittin íìstí_ndaÛô Cadd...",Koôuyolu,"Koôuyolu, ÛÁstanbul",29.041297,41.009847,"World Cuisine, Patisserie, Cafe",1,0,0,0,0,0,105,Turkish Lira(TL),TRY,25.950916,No,No,No,No,3,4.2,4,4,1034
9548,5915807,Huqqa,208,ÛÁstanbul,"Kuruí_eôme Mahallesi, Muallim Naci Caddesi, N...",Kuruí_eôme,"Kuruí_eôme, ÛÁstanbul",29.034640,41.055817,"Italian, World Cuisine",1,0,0,1,0,0,170,Turkish Lira(TL),TRY,42.015768,No,No,No,No,4,3.7,3,3,661
9549,5916112,Aôôk Kahve,208,ÛÁstanbul,"Kuruí_eôme Mahallesi, Muallim Naci Caddesi, N...",Kuruí_eôme,"Kuruí_eôme, ÛÁstanbul",29.036019,41.057979,Restaurant Cafe,0,0,0,1,0,0,120,Turkish Lira(TL),TRY,29.658189,No,No,No,No,4,4.0,4,4,901
9550,5927402,Walter's Coffee Roastery,208,ÛÁstanbul,"CafeaÛôa Mahallesi, BademaltÛ± Sokak, No 21/B,...",Moda,"Moda, ÛÁstanbul",29.026016,40.984776,Cafe,1,0,0,0,0,0,55,Turkish Lira(TL),TRY,13.593337,No,No,No,No,2,4.0,4,4,591


# Location vs Rating

In [15]:
Country = pd.read_excel('./Country-Code.xlsx')
df = pd.merge(df, Country,on='Country Code')
Temp = df['Country'].value_counts()
data1 = [go.Choropleth(
    locationmode='country names',
    locations=Temp.index.values,
    text=Temp.index,
    z=Temp.values
)]
layout = go.Layout(
    title='Number of Resturants Registered on Zomato by Country',
)

fig = go.Figure(data=data1, layout=layout)
iplot(fig)

In [16]:
data1 = [go.Bar(x=Temp.values,
            y=Temp.index,orientation ='h')]
layout = go.Layout(
    title='Number of Resturants Registered on Zomato by Country',
)

fig = go.Figure(data=data1, layout=layout)
iplot(fig)

In [17]:

Temp=df.groupby('Country').agg({'Restaurant ID':'size', 'Aggregate rating':'mean'}).rename(columns={'Restaurant ID':'count'}).reset_index().sort_values(by=['count', "Aggregate rating"],ascending=False)

data1 = ([go.Choropleth(
    locationmode='country names',
    locations=Temp['Country'],
    text=Temp['Country'],
    z=Temp['Aggregate rating']
)])
layout = go.Layout(
    title='Average Rating on Zomato by Country',
)

fig = go.Figure(data=data1, layout=layout)
iplot(fig)

In [18]:
data1 = ([go.Bar(x=Temp['Country'],
    y=Temp['Aggregate rating'],
    text=Temp['count'],
    textposition="auto")])
layout = go.Layout(
    title='Average on Zomato by Country',
)

fig = go.Figure(data=data1, layout=layout)
iplot(fig)

## Focussing on India now

In [19]:
df_India = df[df['Country']=='India']
Resturants_India = (df_India.groupby(['City'], as_index=False)['Aggregate rating'].count()).sort_values(by='Aggregate rating',ascending=False)
Resturants_India.columns = ['City', 'Number of Resturants']

trace0 = go.Bar(
    x=Resturants_India['City'],
    y=Resturants_India['Number of Resturants'],
    )

data1 = [trace0]
layout = go.Layout(
    title='Number of Resturants on Zomato by City',
)

fig = go.Figure(data=data1, layout=layout)
iplot(fig)

In [20]:
Resturants_India=df_India.groupby('City').agg({'Restaurant ID':'size', 'Aggregate rating':'mean'}).rename(columns={'Restaurant ID':'count','Aggregate rating':'rating mean'}).reset_index().sort_values(by=['count', "rating mean"],ascending=False)


trace0 = go.Bar(
    x=Resturants_India['City'],
    y=Resturants_India['rating mean'],
    text=Resturants_India['count'],
    textposition = 'auto',
    opacity=0.9
    )

data1 = [trace0]
layout = go.Layout(
    title='Average Rating on Zomato by City in India',
)

fig = go.Figure(data=data1, layout=layout)
iplot(fig)

# Mixed Cuisines vs Rating

In [21]:
mixed_cusine_data=(df.groupby(['Cuisines'], as_index=False)['Restaurant ID'].count())
mixed_cusine_data.columns = ['Cuisines', 'Number of Resturants']
mixed_cusine_data['Mean Rating'] = (df.groupby(['Cuisines'], as_index=False)['Aggregate rating'].mean())['Aggregate rating']
mixed_cusine_data['color'] = pd.Series('random text', index=mixed_cusine_data.index)

for index, row in mixed_cusine_data.iterrows():
    mixed_cusine_data.set_value(col='color', index=index, value='rgba(222,45,38,%4.1f)' %(mixed_cusine_data.iloc[index]['Mean Rating']/5))

TwentyMostPopularCusines = (mixed_cusine_data.sort_values(['Number of Resturants'],ascending=False).head(50))['Cuisines']
top50_mixed_cuisine_Restaurants_with_most_rating = mixed_cusine_data.sort_values(['Number of Resturants'],ascending=False).head(50)

In [22]:
trace0 = go.Bar(
    x=top50_mixed_cuisine_Restaurants_with_most_rating['Cuisines'],
    y=top50_mixed_cuisine_Restaurants_with_most_rating['Number of Resturants'],
    marker=dict(
        color=top50_mixed_cuisine_Restaurants_with_most_rating['color']),
    text=top50_mixed_cuisine_Restaurants_with_most_rating['Mean Rating'],
    textposition="auto"
 )

data = [trace0]
layout = go.Layout(
    title='Top 50 Mixed Cuisine vs Rating',
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='color-bar')

# Single Cuisine vs Rating 

In [23]:
df1 = df.copy()
cuisines = df1.Cuisines.str.split(',', expand=True).stack().str.strip().reset_index(level=1, drop=True)
df_cuisines = pd.DataFrame(data=cuisines, columns=["Cuisines"])
df1 = df.drop(['Cuisines'], axis=1).join(df_cuisines).reset_index(drop=True)

single_cusine_data=(df1.groupby(['Cuisines'], as_index=False)['Restaurant ID'].count())
single_cusine_data.columns = ['Cuisines', 'Number of Resturants']
single_cusine_data['Mean Rating']=(df1.groupby(['Cuisines'], as_index=False)['Aggregate rating'].mean())['Aggregate rating']
single_cusine_data['color'] = pd.Series('random text', index=single_cusine_data.index)

for index, row in single_cusine_data.iterrows():
    single_cusine_data.set_value(col='color', index=index, value='rgba(222,45,38,%4.1f)' %(single_cusine_data.iloc[index]['Mean Rating']/5))
top50_cuisine_names = (single_cusine_data.sort_values(['Number of Resturants'],ascending=False).head(50))['Cuisines']
top50_single_cuisine = single_cusine_data.sort_values(['Number of Resturants'],ascending=False).head(50)

In [24]:
trace0 = go.Bar(
    x=top50_single_cuisine['Cuisines'],
    y=top50_single_cuisine['Number of Resturants'],
    marker=dict(
        color=top50_single_cuisine['color']),
    text=top50_single_cuisine['Mean Rating'],
    textposition="auto"
 )

data = [trace0]
layout = go.Layout(
    title='Top 50 Single Cuisine vs Rating',
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='color-bar')

In [25]:
df_top50_popular_cusines = (df1[df1['Cuisines'].isin(top50_cuisine_names)])
top50_popular_cusines = top50_cuisine_names.reset_index()['Cuisines']

a=[None] * 50
b= [None] * 50
trace = [None] * 50
for i in range (0,50):
    top50_popular_cusines[i]
    a[i]= df_top50_popular_cusines[df_top50_popular_cusines['Cuisines']==top50_popular_cusines[i]]['Aggregate rating']
    b[i] =a[i].reset_index()['Aggregate rating']
    trace[i] = go.Box(
    y=b[i],
    name = top50_popular_cusines[i],
        boxmean=True
    )
data1 = trace
layout = go.Layout(
    title='How are these 50 Cuisines rated ',
)

fig = go.Figure(data=data1, layout=layout)
iplot(fig)

# Price Range vs Rating

In [26]:
df_price_range = df.copy()

price_range_data=(df_price_range.groupby(['Price range'], as_index=False)['Restaurant ID'].count())
price_range_data.columns = ['Price range', 'Number of Resturants']
price_range_data['Mean Rating']=(df.groupby(['Price range'], as_index=False)['Aggregate rating'].mean())['Aggregate rating']
price_range_data

,Price range,Number of Resturants,Mean Rating
0,1,4444,1.999887
1,2,3113,2.941054
2,3,1408,3.683381
3,4,586,3.817918


In [27]:
price_range_data['color'] = pd.Series('random text', index=price_range_data.index)

for index, row in price_range_data.iterrows():
    price_range_data.set_value(col='color', index=index, value='rgba(222,45,38,%4.1f)' %(price_range_data.iloc[index]['Mean Rating']/5))

trace0 = go.Bar(
    x=price_range_data['Price range'],
    y=price_range_data['Mean Rating'],
 )

data = [trace0]
layout = go.Layout(
    title='Price Range vs Rating',
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='color-bar')

#### Observation : as the price range increases the rating increases.

# Average Cost for two vs Rating

In [28]:
df_cost = df.copy()

rating_grouped_data=(df_cost.groupby(['Aggregate rating'], as_index=False)['Restaurant ID'].count())
rating_grouped_data.columns = ['Aggregate rating', 'Number of Resturants']
rating_names = (rating_grouped_data.sort_values(['Aggregate rating'],ascending=False).head(50))['Aggregate rating']

df_top_ratings = (df_cost[df_cost['Aggregate rating'].isin(rating_names)])
top_ratings = rating_names.reset_index()['Aggregate rating']

a=[None] * rating_names.size
b= [None] * rating_names.size
trace = [None] * rating_names.size
for i in range (0, rating_names.size):
    top_ratings[i]
    a[i]= df_top_ratings[df_top_ratings['Aggregate rating']==top_ratings[i]]['Currency in USD']
    b[i] =a[i].reset_index()['Currency in USD']
    trace[i] = go.Box(
    y=b[i],
    name = top_ratings[i],
        boxmean=True
    )
data1 = trace
layout = go.Layout(
    title='Average price for two(US Dollars) vs Rating',
)

fig = go.Figure(data=data1, layout=layout)
iplot(fig)

#### Its obvious higher rating restaurants offer costlier food. in other words price for two increases as rating of an restaurant increases.

# Votes vs Rating

In [29]:
df_votes = df.query('Votes>20')

rating_grouped_data=(df_votes.groupby(['Aggregate rating'], as_index=False)['Restaurant ID'].count())
rating_grouped_data.columns = ['Aggregate rating', 'Number of Resturants']
rating_names = (rating_grouped_data.sort_values(['Aggregate rating'],ascending=False).head(50))['Aggregate rating']

df_top_ratings = (df_cost[df_cost['Aggregate rating'].isin(rating_names)])
top_ratings = rating_names.reset_index()['Aggregate rating']

a=[None] * rating_names.size
b= [None] * rating_names.size
trace = [None] * rating_names.size
for i in range (0, rating_names.size):
    top_ratings[i]
    a[i]= df_top_ratings[df_top_ratings['Aggregate rating']==top_ratings[i]]['Votes']
    b[i] =a[i].reset_index()['Votes']
    trace[i] = go.Box(
    y=b[i],
    name = top_ratings[i],
        boxmean=True
    )
data1 = trace
layout = go.Layout(
    title='Votes vs Rating',
)

fig = go.Figure(data=data1, layout=layout)
iplot(fig)

#### Its clear restaurant that are rated less receive less votes, on the contrary rating increases with votes.

# OLS for predicting rating

In [30]:
df_model = df[['Currency in USD', "Votes", "Has Table booking", "Has Online delivery", 'c_others', 'c_middle_eastern', 'c_latin', 'c_continental', 'c_asian','c_indian', "Price range", 'Rating color']].copy()

df_model['Has Table booking'] = pd.get_dummies(df_model["Has Table booking"])["Yes"]
df_model['Has Online delivery'] = pd.get_dummies(df_model["Has Online delivery"])["Yes"]

df_model.columns = ["PriceInUSD", "Votes", "HasTableBooking", "HasOnlineDelivery",'c_others', 'c_middle_eastern', 'c_latin', 'c_continental', 'c_asian','c_indian', "PriceRange", "RatingColor"]

feature_data = df_model.drop("RatingColor", axis=1)
target_data = df_model["RatingColor"]
reg = linear_model.LinearRegression()  # this is the sklearn method
reg.fit(feature_data, target_data)
print(reg.coef_)
print("Intercept is %.4f" % reg.intercept_)

r2 = reg.score(feature_data, target_data)
r2_adj = 1 - (1-r2)*(len(feature_data)-1) / (len(target_data) - feature_data.shape[1]-1)

print("R-squared adjusted for the full model is %.5f" % r2_adj)
olsScore = round(r2*100, 2)
print("olsScore", olsScore)

[ 0.0072492   0.00080512 -0.26699366  0.43776959  0.23911426  0.20436343
  0.50934723  0.1452857   0.00114156 -0.19242662  0.52716275]
Intercept is 0.7897
R-squared adjusted for the full model is 0.36355
olsScore 36.43


# Backward Propogation

In [31]:
baseline_performance = r2_adj

for feature in feature_data.columns:
    feature_data_temp = feature_data.drop(feature, axis=1)
    reg_up = linear_model.LinearRegression()
    reg_up.fit(feature_data_temp, target_data)
    
    r2_up = reg_up.score(feature_data_temp, target_data)
    r2_adj_up = 1 - (1-r2_up)*(len(target_data)-1) / (len(target_data) - feature_data_temp.shape[1]-1)

    if r2_adj_up > baseline_performance:
        print("We can improve R-squared to %.5f by leaving out %s" % (r2_adj_up, feature))
    print(r2_adj_up)

0.356828076176
0.306571577364
0.360675705002
0.344695112339
0.359220176243
0.363272289658
0.360409176382
0.36134732221
We can improve R-squared to 0.36362 by leaving out c_asian
0.363621025632
0.360309660564
0.30560527829


### removing c_asian as a feature

In [32]:
df_model = df[['Currency in USD', "Votes", "Has Table booking", "Has Online delivery", 'c_others', 'c_middle_eastern', 'c_latin', 'c_continental', 'c_indian', "Price range", 'Rating color']].copy()

df_model['Has Table booking'] = pd.get_dummies(df_model["Has Table booking"])["Yes"]
df_model['Has Online delivery'] = pd.get_dummies(df_model["Has Online delivery"])["Yes"]

df_model.columns = ["PriceInUSD", "Votes", "HasTableBooking", "HasOnlineDelivery",'c_others', 'c_middle_eastern', 'c_latin', 'c_continental','c_indian', "PriceRange", "RatingColor"]

feature_data = df_model.drop("RatingColor", axis=1)
target_data = df_model["RatingColor"]
reg = linear_model.LinearRegression()  # this is the sklearn method
reg.fit(feature_data, target_data)
print(reg.coef_)
print("Intercept is %.4f" % reg.intercept_)

r2 = reg.score(feature_data, target_data)
r2_adj = 1 - (1-r2)*(len(feature_data)-1) / (len(target_data) - feature_data.shape[1]-1)

print("R-squared adjusted for the full model is %.5f" % r2_adj)
olsScore = round(r2*100, 2)
print("olsScore", olsScore)

[ 0.00724854  0.0008051  -0.26684642  0.43783373  0.23882024  0.20414895
  0.50928204  0.14521668 -0.1923976   0.52726147]
Intercept is 0.7899
R-squared adjusted for the full model is 0.36362
olsScore 36.43


# Forward Selection

In [33]:
baseline_performance = 0
feature_pool = feature_data.columns.tolist()

for feature in feature_pool:
    feature_data_temp = pd.DataFrame(feature_data[feature])
    reg_up = linear_model.LinearRegression()
    reg_up.fit(feature_data_temp, target_data)
    
    r2_up = reg_up.score(feature_data_temp, target_data)
    r2_adj_up = 1 - (1-r2_up)*(len(target_data)-1) / (len(target_data) - feature_data_temp.shape[1]-1)    
    
    if r2_adj_up > baseline_performance:
        print("We can improve R-squared to %.5f by including %s" % (r2_adj_up, feature))   

We can improve R-squared to 0.12720 by including PriceInUSD
We can improve R-squared to 0.15990 by including Votes
We can improve R-squared to 0.03733 by including HasTableBooking
We can improve R-squared to 0.03105 by including HasOnlineDelivery
We can improve R-squared to 0.01465 by including c_others
We can improve R-squared to 0.00751 by including c_middle_eastern
We can improve R-squared to 0.02312 by including c_latin
We can improve R-squared to 0.04300 by including c_continental
We can improve R-squared to 0.02891 by including c_indian
We can improve R-squared to 0.23919 by including PriceRange


In [34]:
corr_matrix = feature_data.corr()
corr_matrix

,PriceInUSD,Votes,HasTableBooking,HasOnlineDelivery,c_others,c_middle_eastern,c_latin,c_continental,c_indian,PriceRange
PriceInUSD,1.000000,0.183067,0.283379,-0.051657,-0.002929,0.060761,0.116604,0.142395,-0.121741,0.595116
Votes,0.183067,1.000000,0.169422,0.074303,0.042309,0.036786,0.103432,0.144437,-0.035991,0.309444
HasTableBooking,0.283379,0.169422,1.000000,0.101224,-0.070043,0.018296,0.054886,0.078562,0.097489,0.501925
HasOnlineDelivery,-0.051657,0.074303,0.101224,1.000000,-0.008668,0.011807,0.001073,0.051588,-0.002644,0.077918
c_others,-0.002929,0.042309,-0.070043,-0.008668,1.000000,0.030372,-0.003991,0.089639,-0.480667,-0.025454
c_middle_eastern,0.060761,0.036786,0.018296,0.011807,0.030372,1.000000,0.059834,0.030881,-0.050387,0.118270
c_latin,0.116604,0.103432,0.054886,0.001073,-0.003991,0.059834,1.000000,0.103988,-0.083122,0.136477
c_continental,0.142395,0.144437,0.078562,0.051588,0.089639,0.030881,0.103988,1.000000,-0.309865,0.192090
c_indian,-0.121741,-0.035991,0.097489,-0.002644,-0.480667,-0.050387,-0.083122,-0.309865,1.000000,-0.035952
PriceRange,0.595116,0.309444,0.501925,0.077918,-0.025454,0.118270,0.136477,0.192090,-0.035952,1.000000


In [35]:
trace = go.Heatmap(z=np.array(corr_matrix), x=corr_matrix.columns, y=corr_matrix.columns)
data=[trace]
iplot(data)

# Using Other Classification Algorithms

In [36]:
df_model_rating_color = df[['Currency in USD', "Votes", "Has Table booking", "Has Online delivery", 'c_others', 'c_middle_eastern', 'c_latin', 'c_continental', 'c_asian','c_indian', "Price range", 'Rating color']].copy()

df_model_rating_color['Has Table booking'] = pd.get_dummies(df_model_rating_color["Has Table booking"])["Yes"]
df_model_rating_color['Has Online delivery'] = pd.get_dummies(df_model_rating_color["Has Online delivery"])["Yes"]

df_model_rating_color.columns = ["PriceInUSD", "Votes", "HasTableBooking", "HasOnlineDelivery",'c_others', 'c_middle_eastern', 'c_latin', 'c_continental', 'c_asian','c_indian', "PriceRange", "RatingColor"]

X_train = df_model_rating_color.drop("RatingColor", axis=1)
Y_train = df_model_rating_color["RatingColor"]

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier


# Logistic Regression

logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)

# Support Vector Machines

svc = SVC()
svc.fit(X_train, Y_train)
acc_svc = round(svc.score(X_train, Y_train) * 100, 2)

# Random Forest

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)

# Decision Tree

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)

# knn

knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)

# Stochastic Gradient Descent

sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
acc_sgd = round(sgd.score(X_train, Y_train) * 100, 2)

# Linear SVC

linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)
acc_linear_svc = round(linear_svc.score(X_train, Y_train) * 100, 2)

# Perceptron

perceptron = Perceptron()
perceptron.fit(X_train, Y_train)
acc_perceptron = round(perceptron.score(X_train, Y_train) * 100, 2)

# Gaussian Naive Bayes

gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
acc_gaussian = round(gaussian.score(X_train, Y_train) * 100, 2)




/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning:

max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning:

max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.



In [37]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'k-Nearest Neighbors', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 'Linear Support Vector Machine', 
              'Decision Tree', "Ordinary Least Squares"],
    'Score': [acc_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_decision_tree, olsScore]})
models.sort_values(by='Score', ascending=False)

,Model,Score
3,Random Forest,98.70
8,Decision Tree,98.70
0,Support Vector Machines,84.85
1,k-Nearest Neighbors,80.83
2,Logistic Regression,69.94
4,Naive Bayes,62.88
7,Linear Support Vector Machine,61.71
5,Perceptron,38.85
6,Stochastic Gradient Decent,36.78
9,Ordinary Least Squares,36.43
